# MCA572– Neural Networks and Deep Learning


**Regular lab Question – 7**

*Priya Dharshini G - 2347247*

Lab Assignment: LSTM Lab Exercise: Poem Generation
Objective

To build, train, and evaluate an LSTM-based text generation model to generate new lines
of poetry. The goal is to understand sequence modeling for creative text generation.

Import Required Libraries

In [17]:
import numpy as np
import pandas as pd
import string
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam

1. Dataset Preparation:
- Download the dataset from Kaggle.
- Load the dataset and explore the columns to understand the structure.
- Concatenate multiple poems into a single text corpus, separating them by
newline characters for clarity.

In [18]:
# Load the dataset
data = pd.read_csv('/content/PoetryFoundationData.csv')
# Concatenate poems into a single text corpus
corpus = "\n".join(data['Poem'].values)

# Limit the dataset to an even smaller size for better memory management
lines = corpus.split("\n")[:500]  # Use only the first 500 lines
corpus_trimmed = "\n".join(lines)

# If you need to restrict by word count, do so here
words = corpus_trimmed.split()[:5000]  # Limit to the first 5,000 words
corpus_trimmed = " ".join(words)

**Dataset Preparation and Trimming**

Explanation: We start by loading and trimming the dataset. The goal is to reduce the corpus size to avoid memory overload. By taking only the first 500 lines and limiting it further to 5,000 words, we create a smaller and more manageable text corpus.

**Working:**

- corpus.split("\n")[:500] splits the corpus by lines and selects only the first 500 lines.
- corpus_trimmed.split()[:5000] splits the trimmed corpus into words and keeps only the first 5,000 words.
The trimmed corpus is joined back into a single text structure.

**Purpose:**

This aggressively reduces the size of the dataset to ensure that we don’t run out of memory during preprocessing or model training.

2. Data Preprocessing:
- Convert the text to lowercase and remove special characters or
punctuation if necessary.
- Tokenize the text (e.g., convert each word to a unique integer).
- Use a sliding window to create sequences of words for the LSTM model.

For example, if n=5, create sequences of 5 words with the 6th word as the
target.
- Pad the sequences so that they all have the same length.

In [19]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Tokenize and create sequences with reduced max length for memory efficiency
tokenizer = Tokenizer()
tokenizer.fit_on_texts([corpus_trimmed])
total_words = len(tokenizer.word_index) + 1

# Create sequences with a shorter max length
max_sequence_len = 10  # Shorter max sequence length to reduce padding
input_sequences = []

# Generate tokenized sequences with the trimmed dataset
for line in corpus_trimmed.split("\n"):
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

# Pad sequences with reduced padding length
input_sequences = pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre')

# Separate predictors and labels
predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
label = to_categorical(label, num_classes=total_words)

**Tokenization and Sequence Preparation**

Explanation: Tokenization is essential to convert words into numerical values for model processing. We create sequences from the text, using a sliding window technique to capture small chunks of text.

**Working:**

- Tokenizer() initializes the tokenizer, which assigns a unique integer to each word.
- tokenizer.fit_on_texts([corpus_trimmed]) fits the tokenizer on the trimmed text corpus, creating a vocabulary index.
- total_words = len(tokenizer.word_index) + 1 calculates the total vocabulary size.
- We create sequences for each line using a loop and append them to input_sequences.
- pad_sequences ensures each sequence is the same length (defined by max_sequence_len) by adding padding at the beginning.

**Purpose:**

This step prepares data for the LSTM model by creating consistent sequences that capture patterns in the text, and padding ensures that all sequences have the same shape for efficient processing.



3. LSTM Model Development:
- Define an LSTM model with the following structure:
1. An embedding layer with an appropriate input dimension (based
on vocabulary size) and output dimension (e.g., 100).
2. One or two LSTM layers with 100 units each.
3. A dropout layer with a rate of 0.2 to prevent overfitting.
4. A dense output layer with softmax activation for word prediction.

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

# Define and compile the model
model = Sequential()
model.add(Embedding(total_words, 50, input_length=max_sequence_len-1))  # Reduced embedding size
model.add(LSTM(50, return_sequences=True))  # Smaller LSTM layer to reduce memory
model.add(Dropout(0.1))  # Lower dropout rate
model.add(LSTM(50))
model.add(Dense(total_words, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


**Model Architecture**


Explanation: We define an LSTM model with a simple structure, optimized for memory efficiency. The architecture includes an embedding layer, two LSTM layers, a dropout layer, and a dense output layer.

**Working:**

- Embedding(total_words, 50, input_length=max_sequence_len-1) creates an embedding layer that transforms each word into a 50-dimensional vector.
- LSTM(50, return_sequences=True) adds the first LSTM layer with 50 units and returns the full sequence to the next layer.
- Dropout(0.1) randomly drops 10% of the connections in the layer to prevent overfitting.
- LSTM(50) adds a second LSTM layer with 50 units, which helps the model capture complex patterns.
- Dense(total_words, activation='softmax') is the output layer that uses softmax activation to predict the next word in the sequence.

**Purpose:**

This simplified model architecture is designed to capture patterns in text while remaining memory-efficient.

4. Training:
- Compile the model with categorical cross-entropy as the loss function
and accuracy as the metric.
- Train the model on the sequences for 10-20 epochs (or until it achieves
satisfactory performance).

In [14]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
history = model.fit(predictors, label, epochs=20, verbose=1)


Epoch 1/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 6s 15ms/step - accuracy: 0.0486 - loss: 6.9692
Epoch 2/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 20ms/step - accuracy: 0.0641 - loss: 6.2106
Epoch 3/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - accuracy: 0.0592 - loss: 6.2177
Epoch 4/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 17ms/step - accuracy: 0.0640 - loss: 6.1011
Epoch 5/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.0610 - loss: 6.0606
Epoch 6/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - accuracy: 0.0602 - loss: 5.9095
Epoch 7/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 3s 17ms/step - accuracy: 0.0638 - loss: 5.7559
Epoch 8/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - accuracy: 0.0664 - loss: 5.6865
Epoch 9/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 4s 15ms/step - accuracy: 0.0584 - loss: 5.6500
Epoch 11/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.0636 - loss: 5.6123
Epoch 12/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 2s 15ms/step - accuracy: 0.0659 - loss: 5.4874
Epoch 13/20
106/106 ━━━━━━━━━━━━━━━━━━━━ 

**Model Compilation and Training**

Explanation: The model is compiled with categorical_crossentropy as the loss function and adam optimizer, and is trained in small batches to save memory.

**Working:**

- loss='categorical_crossentropy' is used for multi-class classification, as each word in the vocabulary is treated as a class.
- optimizer='adam' is a popular optimizer for text data, improving convergence.
- batch_size=16 ensures only 16 sequences are processed in memory at once, helping with memory management.

**Purpose:**

Compiling and training the model in small batches allows the model to learn patterns in the text data without exhausting available memory.



5. Text Generation:
- After training, write a function to generate new poetry lines:
1. Start with a seed text (e.g., a short phrase).
2. Predict the next word, append it to the seed text, and use this
updated text to predict the following word.
3. Repeat this process for a specified number of words or lines.
- Generate multiple lines of poetry using different starting phrases.

In [15]:
def generate_poetry(seed_text, next_words=20):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
        predicted = model.predict(token_list, verbose=0)
        predicted_word_index = np.argmax(predicted, axis=-1)[0]
        output_word = tokenizer.index_word[predicted_word_index]
        seed_text += " " + output_word
    return seed_text


**Text Generation**

Explanation: After training, we use the model to generate new lines of poetry. We start with a seed text (a short phrase) and predict the next word repeatedly to form a sequence.

**Working:**

- Tokenizing the Seed Text: We convert the starting phrase (seed text) into tokens using tokenizer.texts_to_sequences().
- Padding the Token List: We pad the tokenized list to match the input shape required by the model.
- Predicting the Next Word: The model’s output gives a probability distribution over the vocabulary for the next word. np.argmax(predicted, axis=-1)[0] picks the word with the highest probability.
- Updating the Seed Text: The predicted word is appended to the seed text, and the updated text is used to predict the next word. This repeats until the specified number of words is generated.


**Purpose:**

This function generates poetry by iteratively building on a seed phrase, using the learned patterns to predict and add each new word in sequence.

6. Evaluation and Experimentation:
- Experiment with different LSTM layer sizes, dropout rates, and sequence
lengths to observe their effects on generated text quality.
- Try adding additional LSTM layers and tuning hyperparameters to improve
the creativity or fluency of generated poetry.

In [16]:
print(generate_poetry("The sun rises", next_words=30))


The sun rises clack life less between in the mother of were air in the road of the mother of the mother of the forgotten of the mother of the mother of the


**Experimentation and Evaluation**
Explanation: This step involves experimenting with model parameters (e.g., number of LSTM layers, dropout rate, batch size, etc.) to understand their impact on text generation quality and model performance. After training and tuning, the model is evaluated based on the fluency, coherence, and creativity of the generated poetry.

**Possible Experiments:**

1. Varying LSTM Layers and Units: Trying models with one vs. two LSTM layers, or increasing units from 50 to 100, to see if this improves the generated text.
2. Adjusting Dropout Rate: Higher dropout rates (e.g., 0.3 instead of 0.1) can prevent overfitting, while lower rates may lead to more consistent results.
3. Testing Different Sequence Lengths: Trying shorter or longer input sequences to assess if it helps in generating more meaningful lines.
4. Hyperparameter Tuning: Experiment with different values for learning rate, embedding size, and batch size to balance training time and model performance.



**Purpose:**

Experimentation and evaluation allow you to fine-tune the model and gauge how different configurations impact the quality of generated poetry.

Through the dataset trimming, efficient preprocessing, and carefully structured LSTM architecture, the model successfully generates poetry without exhausting memory. Each part of the code is tailored to maximize learning while managing memory, making it suitable for limited-resource environments. The final poetry generation step produces creative text sequences, and experimentation helps refine the model to improve output quality. This approach provides a balanced framework for handling text generation in constrained computational setups, achieving the objectives of poetic structure and stylistic resemblance.